This code is for a 3 state markov chain

LOAD THE LIBRARIES

In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

DEFINE THE GENERATOR AND DISCRIMINATOR MODEL ARCHITECHTURE

In [34]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(100,), activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    # model.add(layers.Dense(1024, activation='relu'))
    # model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(20, activation='tanh'))
    return model

In [37]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(20,), activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

DEFINE THE LOSSES

In [40]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [41]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [42]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

DEFINE THE OPTIMIZERS

In [43]:
# Define the Generator optimizer
generator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)

In [44]:

# Define the Discriminator optimizer
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)

THIS IS THE TRAIN STEP FUNCTION

In [45]:
# Define the training function
@tf.function
def train_step(real_sequences):

    # GENERATE LATENT VARIABLE
    noise = tf.random.normal([real_sequences.shape[0], 100])

    # CALCULATE THE LOSSES
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_sequences = generator(noise, training=True)

        real_output = discriminator(real_sequences, training=True)
        fake_output = discriminator(generated_sequences, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # CALCULATE THE GRADIENTS
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # UPDATE THE GENERATOR AND DISCRIMINATOR
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [46]:
# Define the number of training epochs and batch size
num_epochs = 500
batch_size = 100

GENERATE THE TRAINING DATA

In [47]:
# Define the training dataset
# Replace `training_data` with your own training dataset of shape (num_samples, 20)
training_data = np.random.randint(0, 10, size=(1000, 20))
print(training_data.shape)

(1000, 20)


In [48]:
#declare a transition matrix

n = 3
values = [[0, 1, 0], [0 , 0 , 1] , [1 , 0 , 0]]

matrix = np.array(values)

print(matrix)

cumm_matrix = np.array([[0, 1, 1], [0, 0 ,1], [1, 1 , 1]])

print(cumm_matrix)

[[0 1 0]
 [0 0 1]
 [1 0 0]]
[[0 1 1]
 [0 0 1]
 [1 1 1]]


In [49]:
import random

array = np.zeros(200000)  
random_array = np.random.rand(199999)
array[0] = 1
cnt1 = 0
cnt2 = 0

for p in range(199999):
    val = int(array[p])
    x = random_array[p]

    fin = 0

    val = val + 1
        
    if(x <= cumm_matrix[val][0]):
        fin = -1
    elif(x <= cumm_matrix[val][1]): 
        fin = 0
    else:
        fin = 1

    

    array[p+1] = fin


print(cnt1)
print(cnt2)

0
0


In [50]:
trans = np.zeros((3,3))

for p in range(199999):
    val1 = int(array[p]) + 1
    val2 = int(array[p+1]) + 1

    trans[val1][val2] = trans[val1][val2] + 1

for p in range(3):
    sum = 0
    for q in range(3):
        sum = sum + trans[p][q]

    for q in range(3):
        trans[p][q] = trans[p][q]/sum





print(matrix)

[[0 1 0]
 [0 0 1]
 [1 0 0]]


In [51]:
training_data = array.reshape(10000, 20)

In [52]:
print(training_data)

[[ 1. -1.  0. ...  0.  1. -1.]
 [ 0.  1. -1. ... -1.  0.  1.]
 [-1.  0.  1. ...  1. -1.  0.]
 ...
 [ 0.  1. -1. ... -1.  0.  1.]
 [-1.  0.  1. ...  1. -1.  0.]
 [ 1. -1.  0. ...  0.  1. -1.]]


In [53]:
print(training_data.shape)

(10000, 20)


In [54]:
# Create TensorFlow Dataset from the training data
train_dataset = tf.data.Dataset.from_tensor_slices(training_data).shuffle(len(training_data)).batch(batch_size)


In [55]:
print(train_dataset)

<BatchDataset element_spec=TensorSpec(shape=(None, 20), dtype=tf.float64, name=None)>


INITIALIZE THE GENERATOR AND DISCRIMINATOR MODELS

In [56]:
# Instantiate the Generator and Discriminator models
generator = make_generator_model()
discriminator = make_discriminator_model()

HERE WE ARE TRAINING

In [57]:

# Training loop
for epoch in range(num_epochs):
    for sequences in train_dataset:
        train_step(sequences)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} completed.")


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Epoch 10/500 completed.
Epoch 20/500 completed.
Epoch 30/500 completed.
Epoch 40/500 completed.
Epoch 50/500 completed.
Epoch 60/500 completed.
Epoch 70/500 completed.
Epoch 80/500 completed.
Epoch 90/500 completed.
Epoch 100/500 completed.
Epoch 110/500 completed.
Epoch 120/500 completed.
Epoch 130/500 completed.
Epoch 140/500 completed.
Epoch 150/500 completed.
Epoch 160/500 completed.
Epoch 170/500 completed.
Epoch 180/500 completed.
Epoch 190/500 completed.
Epoch 200/500 completed.
Epoch 210/500 completed.
Epoch 220/500 completed.
Epoch 230/500 completed.
Epoch 240/500 completed.
Epoch 250/500 completed.
Epoch 260/500 completed.
Epoch 270/500 completed.
Epoch 280/500 completed.
Epoch 290/500 completed.
Epoch 300/500 completed.
Epoch 310/500 completed.
Epoch 320/500 completed.
Epoch 330/500 completed.
Epoch 340/500 completed.
Epoch 350/500 completed.
Epoch 360/500 completed.
Epoch 370/500 completed.
Epoch 380/500 completed.
Epoch 390/500 completed.
Epoch 400/500 completed.
Epoch 410

In [58]:
# Generate a sequence of length 20
noise = tf.random.normal([1, 100])
generated_sequence = generator(noise, training=False).numpy().squeeze()

print("Generated Sequence:", generated_sequence)


Generated Sequence: [-0.00296721  0.9984574  -0.99874353 -0.00550565  0.9994096  -0.9983182
  0.01371708  0.9992208  -0.9988141  -0.00219619  0.99920315 -0.99919426
  0.01000979  0.99931103 -0.9991055  -0.00120203  0.9992541  -0.9994422
 -0.00462942  0.9994129 ]


GENERATING SEQUENCES AND FINDING OUT THE OUTPUT TRANSITION MATRIX

In [62]:
final = np.zeros((3,3))

for q in range(5000):
    # LATENT VARIABLE
    noise = tf.random.normal([1, 100])

    # USING THE GENERATOR TO GENERATE
    generated_sequence = generator(noise, training=False).numpy().squeeze()

    # ROUNDING OFF
    for p in range(20):
        if(generated_sequence[p] < -0.66):
            generated_sequence[p] = 0
        elif(generated_sequence[p] < 0.33):
            generated_sequence[p] = 1
        else:
            generated_sequence[p] = 2

    for p in range(19):
        final[int(generated_sequence[p])][int(generated_sequence[p+1])] += 1


# CALCULATING THE TRANSITION MATRIX

for p in range(3):
    sum = 0
    for q in range(3):
        sum += final[p][q]

    for q in range(3):
        final[p][q] = final[p][q]/sum



In [63]:
from tabulate import tabulate

In [64]:
def print_matrix_as_table(matrix, matrix_name):
    headers = ["State {}".format(i + 1) for i in range(matrix.shape[1])]
    rows = ["State {}".format(i + 1) for i in range(matrix.shape[0])]
    table = tabulate(matrix, headers=headers, showindex=rows, tablefmt="grid")
    print(f"\n{matrix_name}:\n{table}")

# Print matrices as tables with labels
print_matrix_as_table(matrix, "Original Transition Matrix")
print_matrix_as_table(final, "Generated Transition Matrix")



Original Transition Matrix:
+---------+-----------+-----------+-----------+
|         |   State 1 |   State 2 |   State 3 |
+=========+===========+===========+===========+
| State 1 |         0 |         1 |         0 |
+---------+-----------+-----------+-----------+
| State 2 |         0 |         0 |         1 |
+---------+-----------+-----------+-----------+
| State 3 |         1 |         0 |         0 |
+---------+-----------+-----------+-----------+

Generated Transition Matrix:
+---------+-----------+-----------+-----------+
|         |   State 1 |   State 2 |   State 3 |
+=========+===========+===========+===========+
| State 1 |         0 |         1 |         0 |
+---------+-----------+-----------+-----------+
| State 2 |         0 |         0 |         1 |
+---------+-----------+-----------+-----------+
| State 3 |         1 |         0 |         0 |
+---------+-----------+-----------+-----------+
